In [31]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torchvision
from pathlib import Path

In [4]:
!pip install pretrainedmodels

     |████████████████████████████████| 58 kB 556 kB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=41465a37ba631f227111ff8e333b27d362c14ef27c5a1b94979403df5265e20a
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built pretrainedmodels


In [32]:
class IndoorData:
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = Image.open(img_path).convert('RGB')
        label = img_path.parent.name   
        label = torch.tensor(lbl2idx[label])
        if self.transform is not None:
            img = self.transform(img)
        return img, label

In [33]:
import pretrainedmodels
import torch.nn as nn

def Model(pretrained):
    if pretrained:
        model = pretrainedmodels.__dict__['resnet18'](
            pretrained='imagenet')
    else:
        model = pretrainedmodels.__dict__['resnet18'](
            pretrained=None)
    model.last_linear = nn.Sequential(
        nn.Linear(in_features=512, out_features=67),
        nn.Sigmoid())
    return model

In [34]:
from tqdm import tqdm
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

def train(dataloader, model, optimizer, device):
    model.train()
    tr_loss = 0
    tk0 = tqdm(train_dl, desc='Train')
    for step, batch in enumerate(tk0):
        inputs = batch[0]
        target = batch[1]

        inputs = inputs.to(device, dtype=torch.float)
        targets = target.to(device, dtype=torch.long)

        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        tr_loss += loss
        optimizer.step()
    
def evaluate(dataloader, model, device):
    model.eval()
    val_loss = 0
    val_preds = None
    val_labels = None
    tk0 = tqdm(dataloader, desc='Validation')
    
    for step, batch in enumerate(tk0):
        inputs = batch[0]
        targets = batch[1]
        
        inputs = inputs.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.long)
        
        output = model(inputs)
        loss = criterion(output, targets)
        
        val_loss+= loss.item()
        
    return val_loss

In [38]:
bs = 64
epochs=3

p = Path('.')
images = list(p.glob("../input/indoor-scenes-cvpr-2019/indoorCVPR_09/Images/**/*.jpg"))

labels = [img.parent.name for img in images]
labels = set(labels)
lbl2idx = {lbl: idx for idx, lbl in enumerate(labels)}

train_transform = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(),
        transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
        transforms.ToTensor(),
    ]
)

split = int(len(images)*0.8)
train_ds = IndoorData(images[:split], train_transform)
valid_ds = IndoorData(images[split:], train_transform)

train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=54, shuffle=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Model(pretrained=True)
model.to(device)

criterion = nn.CrossEntropyLoss()
plist = [{'params': model.parameters(), 'lr': 1e-3}]
optimizer = torch.optim.Adam(plist, lr=5e-5)

for epoch in range(epochs):
    train(train_dl, model, optimizer, device)
    val_loss = evaluate(valid_dl, model, device)
    print(f'epoch = {epoch}, valid loss = {val_loss}')
torch.save(model.state_dict(), 'model.pt')